We get the number of individuals per region (with at least one information in the catalogs)

In [1]:
import sys

sys.path.append("../")

import pandas as pd
import numpy as np

from dotenv import load_dotenv

load_dotenv()
import os

import sqlite3

DB_PATH = os.getenv("DB_PATH")
DATA_PATH = "data"

conn = sqlite3.connect(DB_PATH)

import numpy as np

In [25]:
# Individuals Regions
df_ind_regions = pd.read_sql_query("SELECT * FROM individuals_regions", conn)

# Years
df_ind = pd.read_sql_query("SELECT * FROM individuals_main_information", conn)
df_ind_year = df_ind[["individual_wikidata_id", "birthyear"]].drop_duplicates()
df_ind_year = df_ind_year.dropna()

temporal_resolution = 10
df_ind_year["decade"] = df_ind_year["birthyear"].apply(lambda x: round(x / temporal_resolution) * temporal_resolution)

df_catalogs_id = pd.read_sql_query("SELECT * FROM individual_identifiers", conn)

df_catalogs = pd.read_sql_query("SELECT * FROM identifiers", conn)
df_catalogs = df_catalogs[['identifiers_wikidata_id', 'country_name']].dropna()

df_cat = pd.merge(df_catalogs_id, df_catalogs, on = 'identifiers_wikidata_id')
df_cat = df_cat[['individual_wikidata_id','individual_name', 'country_name']].drop_duplicates()
df_cat = df_cat.groupby(['individual_name', 'individual_wikidata_id'])['country_name'].count().rename('score').reset_index()
df_cat = df_cat.sort_values('score', ascending=False).reset_index(drop=True)

df = pd.merge(df_ind_regions,df_ind_year, on = 'individual_wikidata_id')
df = pd.merge(df, df_cat, on = 'individual_wikidata_id')
df = df[['individual_wikidata_id', 'region_name', 'decade']].drop_duplicates()
df['score'] = 1

df = df.groupby(['region_name', 'decade'])['score'].sum().reset_index()
df.to_sql('region_score',conn, if_exists = 'replace', index=False)

5363